# Model South Korea Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.data_plots_kr import load_df_feat_kr
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

from my_helpers.model_kr import prepare_data_features_kr
from my_helpers.model_kr import prepare_dataset_kr

from my_helpers.utils import clean_file

import settings

### Definitions

In [2]:
PATH_TO_SAVE_DATA = settings.PATH_TO_SAVE_DATA

from my_helpers.model_kr import PATH_MDL_MULTI_STEP_KR
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_KR 
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_FILE_KR
from my_helpers.model_kr import PATH_SERVERLESS_KR
from my_helpers.model_kr import URL_PREDICT_KR
from my_helpers.model_kr import PAST_HISTORY # days used to predict next values in future
from my_helpers.model_kr import FUTURE_TARGET  # predict 3 days later
from my_helpers.model_kr import STEP 

from my_helpers.data_plots_kr import PATH_DF_FEAT_KR

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model_kr import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")


TRAIN_SPLIT = 263


### helper functions

### load data

In [3]:
# load
df_feat_kr = load_df_feat_kr()
# clean
df_feat_kr = prepare_data_features_kr(df_feat_kr)
# split train/test
df_feat_kr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_kr.shape[0])]
df_feat_kr

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2020-04-03,10062.0,424365.0,174.0,2020-04-03,5.0,0.0,1.0,5.0,1.0,1.0,...,5.000000,10.0,inf,106.0,4.240000,20.0,20.000000,5.0,inf,True
2020-04-04,10156.0,434888.0,177.0,2020-04-04,6.0,0.0,1.0,1.0,0.0,0.0,...,5.000000,10.0,inf,67.0,1.030769,21.0,21.000000,5.0,inf,True
2020-04-05,10237.0,441662.0,183.0,2020-04-05,0.0,3.0,1.0,4.0,0.0,1.0,...,6.000000,9.0,9.000000,60.0,0.789474,21.0,10.500000,8.0,inf,True
2020-04-06,10284.0,447509.0,186.0,2020-04-06,1.0,0.0,2.0,2.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.769231,22.0,7.333333,8.0,inf,True
2020-04-07,10331.0,456654.0,192.0,2020-04-07,2.0,0.0,1.0,1.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.759494,22.0,5.500000,8.0,inf,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31,60739.0,4043541.0,900.0,2020-12-31,4.0,9.0,22.0,23.0,5.0,12.0,...,0.936709,75.0,1.339286,522.0,3.052632,390.0,1.387900,262.0,3.638889,False
2021-01-01,61767.0,4092389.0,917.0,2021-01-01,5.0,5.0,35.0,24.0,6.0,8.0,...,0.895397,76.0,1.310345,515.0,2.668394,381.0,1.233010,240.0,2.474227,False
2021-01-02,62587.0,4123406.0,942.0,2021-01-02,6.0,13.0,37.0,20.0,11.0,11.0,...,0.886076,82.0,1.389831,506.0,2.468293,386.0,1.229299,229.0,1.908333,False


In [4]:
df_feat_kr[df_feat_kr["train"] == False].head(5)

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2020-12-23,52550.0,3673462.0,739.0,2020-12-23,3.0,33.0,29.0,25.0,12.0,21.0,...,1.134259,69.0,0.831325,333.0,3.112150,351.0,1.240283,215.0,10.238095,False
2020-12-24,53533.0,3731677.0,756.0,2020-12-24,4.0,26.0,21.0,50.0,2.0,12.0,...,1.197115,66.0,0.835443,375.0,3.318584,356.0,1.401575,228.0,7.354839,False
2020-12-25,54770.0,3782568.0,773.0,2020-12-25,5.0,20.0,15.0,67.0,5.0,14.0,...,1.340314,67.0,0.930556,439.0,3.884956,359.0,1.581498,243.0,7.363636,False
2020-12-26,55902.0,3812403.0,793.0,2020-12-26,6.0,24.0,22.0,34.0,6.0,37.0,...,1.526596,65.0,0.890411,454.0,3.574803,364.0,1.582609,267.0,8.612903,False
2020-12-27,56872.0,3839436.0,808.0,2020-12-27,0.0,7.0,44.0,34.0,5.0,8.0,...,1.588889,64.0,1.084746,470.0,3.507463,384.0,1.606695,271.0,8.212121,False


### Prepare dataset

In [5]:
dataset, data_std, data_mean = prepare_dataset_kr(df_feat_kr)
print(dataset)
print("data_mean : ", data_mean)
print("data_std : ", data_std)

[[-0.97450826 -0.49350721 -1.2887581  ... -0.21416374  0.99528077
  -0.41215472]
 [-0.9705845  -0.60404409 -1.46231458 ... -0.2118543   1.49292115
  -0.41215472]
 [-1.28056105 -1.03734863 -1.98107417 ... -0.75298622 -1.49292115
  -0.41215472]
 ...
 [-2.22226195 -2.72635207 -1.29186159 ...  2.74625659  1.49292115
   0.22669013]
 [-2.30466078 -2.77498829 -1.97104752 ...  2.73802919 -1.49292115
   0.44381541]
 [-2.38705961 -3.25692906 -0.55060594 ...  2.81741637 -0.99528077
   0.44381541]]
data_mean :  [1.35120406e+01 2.25538657e+01 6.46346641e+01 1.82195184e+01
 1.54429658e+02 1.19907414e+04 3.00000000e+00 4.43632349e+01]
data_std :  [8.49526638e+00 7.53896237e+00 1.39627942e+01 6.06204431e+00
 2.85166288e+02 6.92807025e+03 2.00948322e+00 5.98093301e+00]


In [6]:
dataset.shape[1]

8

In [7]:
PAST_HISTORY

14

In [8]:
dataset.shape[1]

8

### Load model

In [9]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP_KR)

CPU times: user 2.37 s, sys: 146 ms, total: 2.52 s
Wall time: 2.92 s


In [10]:
multi_step_model.inputs[0].dtype

tf.float32

In [11]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_lstm"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/keras_lstm/assets


### Save model TFlite

In [12]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE_KR)

File /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model_20210106_08_43_35.tflite moved!


In [13]:
open(PATH_MDL_MULTI_TFLITE_FILE_KR, "wb").write(tflite_model)

5500

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [14]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[200 - 214]
[207 - 221]
[214 - 228]
[221 - 235]
[228 - 242]
[235 - 249]
[242 - 256]
[249 - 263]
[256 - 270]
9


array([[ 110.66412 ,  115.580246,  118.56093 ,  121.96298 ,  125.25475 ,
         129.04161 ,  134.54053 ,  138.33543 ,  147.86386 ,  160.91809 ,
         177.46422 ,  189.13292 ,  197.61052 ,  205.67947 ,  271.56055 ,
         287.04593 ,  307.41833 ,  331.30585 ,  347.75702 ,  357.24808 ,
         362.8896  ,  325.11945 ,  343.11267 ,  360.3051  ,  378.21387 ,
         393.37393 ,  404.34286 ,  412.6293  ,  462.99106 ,  487.78815 ,
         519.67834 ,  553.55164 ,  579.219   ,  593.2461  ,  602.64185 ,
         665.39575 ,  685.89087 ,  726.8445  ,  771.42554 ,  804.4784  ,
         821.0642  ,  831.3047  ,  820.62146 ,  869.8695  ,  914.1361  ,
         943.71936 ,  965.3086  ,  971.4104  ,  975.1708  ,  902.46533 ,
         931.2157  ,  967.3849  ,  991.71423 , 1012.1183  , 1018.8502  ,
        1025.1606  ,  900.1637  ,  928.31104 ,  944.3738  ,  939.9888  ,
         941.145   ,  941.0459  ,  944.9075  ]], dtype=float32)

#### Future days

In [15]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [16]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [17]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE_KR)

In [18]:
PATH_MDL_MULTI_TFLITE_FILE_KR

'/Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model.tflite'

### Predict reloaded model

#### Past days

In [19]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [20]:
x_multi.shape

(1, 14, 8)

In [21]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [28]:
dataset.shape

(277, 8)

In [29]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[200 - 214]
[207 - 221]
[214 - 228]
[221 - 235]
[228 - 242]
[235 - 249]
[242 - 256]
[249 - 263]
[256 - 270]


In [30]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda_kr.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [31]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [32]:
y_multi_pred

array([[-0.15347373, -0.13623424, -0.12578179, -0.11385173, -0.10230839,
        -0.08902889, -0.0697457 , -0.05643804, -0.02302444,  0.02275314,
         0.0807759 ,  0.12169483,  0.15142345,  0.17971909,  0.41074586,
         0.4650489 ,  0.5364894 ,  0.62025625,  0.67794603,  0.7112286 ,
         0.73101187,  0.5985623 ,  0.66165966,  0.7219488 ,  0.7847498 ,
         0.8379121 ,  0.8763771 ,  0.9054353 ,  1.0820402 ,  1.1689968 ,
         1.280827  ,  1.3996115 ,  1.4896199 ,  1.538809  ,  1.5717573 ,
         1.7918179 ,  1.863689  ,  2.0073018 ,  2.1636355 ,  2.2795427 ,
         2.3377047 ,  2.3736153 ,  2.3361518 ,  2.5088515 ,  2.664082  ,
         2.7678225 ,  2.8435302 ,  2.8649275 ,  2.8781142 ,  2.6231558 ,
         2.723976  ,  2.8508112 ,  2.9361274 ,  3.0076787 ,  3.0312858 ,
         3.0534148 ,  2.6150846 ,  2.71379   ,  2.7701173 ,  2.7547405 ,
         2.758795  ,  2.7584474 ,  2.7719889 ]], dtype=float32)

In [33]:
y_multi_pred_out

array([[-0.15347375, -0.13623425, -0.12578179, -0.11385175, -0.10230839,
        -0.08902891, -0.06974571, -0.056438  , -0.0230244 ,  0.0227532 ,
         0.08077595,  0.12169488,  0.1514235 ,  0.17971914,  0.41074586,
         0.46504891,  0.53648943,  0.6202563 ,  0.67794603,  0.71122861,
         0.73101187,  0.5985623 ,  0.6616596 ,  0.7219488 ,  0.78474987,
         0.83791202,  0.87637705,  0.90543532,  1.08204019,  1.16899669,
         1.28082693,  1.39961147,  1.48961973,  1.53880894,  1.5717572 ,
         1.7918179 ,  1.86368883,  2.00730181,  2.16363549,  2.27954268,
         2.33770466,  2.37361503,  2.33615184,  2.50885177,  2.66408229,
         2.7678225 ,  2.84353018,  2.86492729,  2.87811446,  2.62315583,
         2.72397614,  2.85081124,  2.93612742,  3.00767851,  3.03128552,
         3.05341458,  2.61508465,  2.7137897 ,  2.77011752,  2.75474048,
         2.75879526,  2.75844741,  2.77198911]])

In [34]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [35]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [36]:
y_future_pred_out

array([[2.65488458, 2.72560668, 2.78775334, 2.79731798, 2.82567215,
        2.84909391, 2.87972999]])

In [37]:
y_future_pred

array([[2.6548848, 2.7256072, 2.7877536, 2.7973182, 2.8256721, 2.8490942,
        2.8797302]], dtype=float32)

In [38]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda-kr
    
- Execute : sls deploy -v

In [39]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE_KR}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr\nserverless deploy -v'

In [40]:
open('deploy_serverless_kr.sh', "w").write(str_exe)
os.chmod('deploy_serverless_kr.sh', stat.S_IRWXU)

In [41]:
!cat ./deploy_serverless_kr.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr
serverless deploy -v

In [42]:
!./deploy_serverless_kr.sh

Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/ef4e42eb03bbad46f74fee99a3c01f994e8119e7a557f947779093ab37b248d3_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: Injecting required Python packages to package...
Serverless: Creating Stack...
Serverless: Checking Stack create progress...
CloudFormation - CREATE_IN_PROGRESS - AWS::CloudFormation::Stack - tensorflow-lite-on-aws-lambda-kr-dev
CloudFormation - CREATE_IN_PROGRESS - AWS::S3::Bucket - ServerlessDeploymentBucket
CloudFormation - CREAT

### API AWS real Test

#### Past days

In [22]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[200 - 214]
[207 - 221]
[214 - 228]
[221 - 235]
[228 - 242]
[235 - 249]
[242 - 256]
[249 - 263]
[256 - 270]
status code :  200
[[[-0.15347374975681305, -0.13623425364494324, -0.1257818043231964, -0.11385174840688705, -0.10230839252471924, -0.08902892470359802, -0.06974571198225021]], [[-0.05643804371356964, -0.023024436086416245, 0.022753160446882248, 0.0807759016752243, 0.12169484794139862, 0.15142346918582916, 0.17971910536289215]], [[0.41074591875076294, 0.46504896879196167, 0.5364894866943359, 0.6202563047409058, 0.677946150302887, 0.7112287282943726, 0.7310119867324829]], [[0.5985623002052307, 0.6616596579551697, 0.7219488620758057, 0.7847498655319214, 0.8379120230674744, 0.8763771653175354, 0.90543532371521]], [[1.0820403099060059, 1.1689966917037964, 1.2808270454406738, 1.3996115922927856, 1.4896198511123657, 1.538809061050415, 1.571757197380066]], [[1.7918179035186768, 1.8636888265609741, 2.0073015689849854, 2.163635492324829, 2.2795426845550537, 2.337704658508301, 2.3736150264

In [23]:
len(json_list_list_x)

20902

In [24]:
resp.json()

[[[-0.15347374975681305,
   -0.13623425364494324,
   -0.1257818043231964,
   -0.11385174840688705,
   -0.10230839252471924,
   -0.08902892470359802,
   -0.06974571198225021]],
 [[-0.05643804371356964,
   -0.023024436086416245,
   0.022753160446882248,
   0.0807759016752243,
   0.12169484794139862,
   0.15142346918582916,
   0.17971910536289215]],
 [[0.41074591875076294,
   0.46504896879196167,
   0.5364894866943359,
   0.6202563047409058,
   0.677946150302887,
   0.7112287282943726,
   0.7310119867324829]],
 [[0.5985623002052307,
   0.6616596579551697,
   0.7219488620758057,
   0.7847498655319214,
   0.8379120230674744,
   0.8763771653175354,
   0.90543532371521]],
 [[1.0820403099060059,
   1.1689966917037964,
   1.2808270454406738,
   1.3996115922927856,
   1.4896198511123657,
   1.538809061050415,
   1.571757197380066]],
 [[1.7918179035186768,
   1.8636888265609741,
   2.0073015689849854,
   2.163635492324829,
   2.2795426845550537,
   2.337704658508301,
   2.373615026473999]],
 [[2.

In [25]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [26]:
y_multi_pred_out

array([[-0.15347375, -0.13623425, -0.1257818 , -0.11385175, -0.10230839,
        -0.08902892, -0.06974571, -0.05643804, -0.02302444,  0.02275316,
         0.0807759 ,  0.12169485,  0.15142347,  0.17971911,  0.41074592,
         0.46504897,  0.53648949,  0.6202563 ,  0.67794615,  0.71122873,
         0.73101199,  0.5985623 ,  0.66165966,  0.72194886,  0.78474987,
         0.83791202,  0.87637717,  0.90543532,  1.08204031,  1.16899669,
         1.28082705,  1.39961159,  1.48961985,  1.53880906,  1.5717572 ,
         1.7918179 ,  1.86368883,  2.00730157,  2.16363549,  2.27954268,
         2.33770466,  2.37361503,  2.33615184,  2.50885153,  2.66408229,
         2.7678225 ,  2.84353018,  2.86492729,  2.87811446,  2.62315583,
         2.72397614,  2.85081124,  2.93612742,  3.00767851,  3.03128552,
         3.05341458,  2.61508465,  2.7137897 ,  2.77011752,  2.75474048,
         2.75879478,  2.75844765,  2.77198911]])

In [27]:
y_multi_pred

array([[-0.15347373, -0.13623424, -0.12578179, -0.11385173, -0.10230839,
        -0.08902889, -0.0697457 , -0.05643804, -0.02302444,  0.02275314,
         0.0807759 ,  0.12169483,  0.15142345,  0.17971909,  0.41074586,
         0.4650489 ,  0.5364894 ,  0.62025625,  0.67794603,  0.7112286 ,
         0.73101187,  0.5985623 ,  0.66165966,  0.7219488 ,  0.7847498 ,
         0.8379121 ,  0.8763771 ,  0.9054353 ,  1.0820402 ,  1.1689968 ,
         1.280827  ,  1.3996115 ,  1.4896199 ,  1.538809  ,  1.5717573 ,
         1.7918179 ,  1.863689  ,  2.0073018 ,  2.1636355 ,  2.2795427 ,
         2.3377047 ,  2.3736153 ,  2.3361518 ,  2.5088515 ,  2.664082  ,
         2.7678225 ,  2.8435302 ,  2.8649275 ,  2.8781142 ,  2.6231558 ,
         2.723976  ,  2.8508112 ,  2.9361274 ,  3.0076787 ,  3.0312858 ,
         3.0534148 ,  2.6150846 ,  2.71379   ,  2.7701173 ,  2.7547405 ,
         2.758795  ,  2.7584474 ,  2.7719889 ]], dtype=float32)

In [28]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [29]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[2.6548848152160645, 2.72560715675354, 2.7877538204193115, 2.79731822013855, 2.825672149658203, 2.8490941524505615, 2.879729986190796]]]


In [30]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[2.65488482, 2.72560716, 2.78775382, 2.79731822, 2.82567215,
        2.84909415, 2.87972999]])

In [31]:
y_future_pred

array([[2.6548848, 2.7256072, 2.7877536, 2.7973182, 2.8256721, 2.8490942,
        2.8797302]], dtype=float32)

In [32]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
